In [1]:
import pandas as pd
from pyhanlp import *
import jieba
import numpy as np
from gensim.models import word2vec

df = pd.read_excel('Lawsnote法律黑客松＿判決摘要.xlsx',sheet_name='慰撫金')

# 濾除雜質
for idx, row in df.iterrows():
    for col in df.columns:
        if type(df.iloc[idx][col]) == str:
            value = df.iloc[idx][col]
            value = value.strip()
            value = value.replace('未滿','').replace('不明','').replace('未主張','')
            value = value.replace('目前沒有工作','').replace('目前在監服刑','').replace('在監執行','')
            value = value.replace('無','').replace('目前在監服刑','').replace('在監執行','')
            df.iloc[idx][col] = value 
            
        if col == '年齡':
            if ('nan' in value) or (value == ''): df.iloc[idx][col] = 0
                    
                
                                        
# 留下必要row           
col = [c for c in df.columns if c not in ['法院']]
df = df.drop([0,1])
df = df.reset_index(drop=True)
df = df.dropna(subset=['最後判賠金額'])
df = df[df["犯罪類型"] != 'X']

In [2]:
def education(text):
    if text!= None and type(text) == str:
        try:
            text = text.replace('\n','')
            seg_list = jieba.cut(text)
            seg_list = [s for s in seg_list]

            seg_list_1 = "".join(seg_list[:-1])
            seg_list_2 = seg_list[-1]

            if 'nan' in seg_list_2: return [0,0]
            if seg_list_1 == '': return [seg_list_2]
            return [seg_list_1,seg_list_2]
        except Exception as e:
            return [0,0]
    else: return [0,0]

def study(v):
    level = 0
    if type(v[0]) == str:
        text = v[0]
        if text in ['國小']: level = 0
        elif text in ['初中','國中','國 中']: level =  1
        elif text in ['高中','高職','高工','家商']: level =  2
        elif text in ['大學','空中大學','二專','專科','大專','技術學院']: level =  3
        elif text in ['研究','碩士']: level =  4
        elif text in ['博士']: level =  5
        if len(v)>1:
            if '肄業' in v[1]: return (level-0.5)
            else: return level

    
def extract_age(text):
    if type(text) == str:
        seg_list = jieba.cut(text)
        for s in seg_list:
            if s.isdigit(): return int(s)
    else:
        return text   
    

    
    

# 年齡處理
df['年齡'] = df['年齡'].fillna(0)
df['年齡'] = df['年齡'].apply(extract_age)
age_count = [v for v in df['年齡'] if v > 0]
df['年齡'] = df['年齡'].apply(lambda x : x if x>0 else np.mean(age_count)) # 年齡填充


# 學歷處理  
df['學歷'] = df['學歷'].apply(education)
level_dict = [v[0] for v in df['學歷'] if type(v[0]) == str]
level_dict = ['國小','初中','國中','國 中','高中','高職','高工','大學','空中大學','二專','碩士']           
df['學歷'] = df['學歷'].apply(study)  
study_count = [v for v in df['學歷'] if v > 0]
df['學歷'] = df['學歷'].apply(lambda x : x if x>0 else np.mean(study_count)) # 學歷填充

# 原告年紀處理 
df['原告年紀'] = df['原告年紀'].fillna(0)
hurt_age_count = [int(v) for v in df['原告年紀'] if int(v) > 0]
df['原告年紀'] = df['原告年紀'].apply(lambda x : x if x>0 else np.max(hurt_age_count)) # 原告年紀

# df
df[['職業','產業','階級','學歷','年齡']].tail()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jack\AppData\Local\Temp\jieba.cache
Loading model cost 0.904 seconds.
Prefix dict has been built succesfully.


,職業,產業,階級,學歷,年齡
221,",公車司機",公共汽車客運業,員工,1.954955,36.272727
222,",法師",其他個人服務業,老闆,2.000000,36.272727
223,待業,NaN,NaN,1.954955,18.000000
224,",戲劇製作人","創作及藝術表演業, 藝文服務業",主管,1.954955,36.272727
225,待業,NaN,NaN,1.954955,36.272727


In [3]:
# 數額填充
import operator
employee = pd.read_excel('Lawsnote法律黑客松＿判決摘要.xlsx',sheet_name='薪情平臺匯出資料') # 薪情平臺匯出資料

employee.head()  
employee_money ={} # 薪資排行榜
employee_mapping={} 
for idx, row in employee.iterrows():
    employee_money[list(row)[0]] = list(row)[2]    
    if type(list(row)[1]) == str:
        employee_mapping[list(row)[0]] = list(row)[1].split(',')
    

employee_rank = sorted(employee_money.items(), key=operator.itemgetter(1),reverse=True)
employee_rank = [e[0] for e in employee_rank]          

def fill_money(row):
    if row['數額']: return row['數額']
    else:
        return employee_money[row['產業']]

    
    
# df['數額'] = df.apply(fill_money)
df['階級'] = df['階級'].fillna('待業')
df['產業'] = df['產業'].fillna('待業')
# df['財產種類'] = df['財產種類'].fillna('無財產')
df[['職業','產業','階級','學歷','年齡','數額']].head()

,職業,產業,階級,學歷,年齡,數額
0,"餐飲及溫泉會館,老闆",住宿及餐飲業,老闆,1.954955,36.272727,1000000
2,待業,待業,待業,2.500000,36.272727,52540
3,待業,待業,待業,1.954955,36.272727,36000
4,醫師,醫療保健業,員工,1.954955,36.272727,1914876
5,待業,待業,待業,2.500000,36.272727,0


In [4]:
# 刪除不必要的columns
df = df.drop(["法院", "主文", "職業", "產業", "等級", "財產總額"], axis=1)
# 所有\n化為，
df.replace('\n','，', inplace=True, regex=True)

# 刪除犯罪類型為nan的index
Nanlist = df[df["犯罪類型"].isnull().values==True].index.tolist()
df = df.drop(Nanlist)

df.reset_index(drop=True, inplace=True)

In [5]:
#犯罪類型 one hot encoding
crime_df = pd.get_dummies(df["犯罪類型"])

In [6]:
#階級 one hot encoding
level_df = pd.get_dummies(df["階級"])

In [7]:
# 財產種類 one hot encoding
t_investment = 0   # 投資
t_real_estate = 0  # 不動產
t_car = 0          # 汽車
t_house = 0        # 房屋
t_land = 0         # 土地
t_motorcycle = 0   # 機車

investment = []   # 投資
real_estate = []  # 不動產
car = []          # 汽車
house = []        # 房屋
land = []         # 土地
motorcycle = []   # 機車

for index in df["財產種類"]:
    if not pd.isnull(index):
        comma = index.split(",")
        for item in comma:
            temp = item.split(":")
            if temp[1] == "n":        # n設定為5
                temp[1] = 5
            
            if temp[0] == "投資":
                t_investment = temp[1]
            elif temp[0] == "不動產":
                t_real_estate = temp[1]
            elif temp[0] == "汽車":
                t_car = temp[1]
            elif temp[0] == "房屋":
                t_house = temp[1]
            elif temp[0] == "土地":
                t_land = temp[1]
            elif temp[0] == "機車":
                t_motorcycle = temp[1]
        investment.append(t_investment)
        real_estate.append(t_real_estate)
        car.append(t_car)
        house.append(t_house)
        land.append(t_land)
        motorcycle.append(t_motorcycle)
        
        t_investment = 0   # 投資
        t_real_estate = 0  # 不動產
        t_car = 0          # 汽車
        t_house = 0        # 房屋
        t_land = 0         # 土地
        t_motorcycle = 0   # 機車
    else:
        investment.append(0)
        real_estate.append(0)
        car.append(0)
        house.append(0)
        land.append(0)
        motorcycle.append(0)

c = {"投資":investment,
     "不動產":real_estate,
     "汽車":car,
     "房屋":house,
     "土地":land,
     "機車":motorcycle}
belongings = pd.DataFrame(c)

In [8]:
# 刪除columns(犯罪類型、階級、財產種類、原告年紀)
df = df.drop(["犯罪類型", "階級", "財產種類", "原告年紀"], axis=1)

In [32]:
df = df.drop(['原告主張','被告主張','有利原告','有利被告'], axis=1)

In [10]:
#原告主張編碼
hack_seg_list = []
analyer = PerceptronLexicalAnalyzer()
for i in df['原告主張']:
    if type(i) == str:
        seg_list = analyer.segment(i)
        word_list = []
        for word_ in seg_list:
            word_list.append(word_)
        word_str = ' '.join(word_list)
        if len(word_list) == 0:
            hack_seg_list.append('NaN')
        else:
            hack_seg_list.append(word_str)
        hack_seg_list.append(len(word_list))
    else:
        hack_seg_list.append('NaN')
        hack_seg_list.append(0)

#news.columns = ['label_num','label_name','content']
#print(len(news))
hack_seg = pd.DataFrame(np.array(hack_seg_list).reshape(int(len(hack_seg_list)/2),2))
hack_seg.columns = ['seg_word','word_count']

In [11]:
#被告主張編碼
hack_seg_list = []
analyer = PerceptronLexicalAnalyzer()
for i in df['被告主張']:
    if type(i) == str:
        seg_list = analyer.segment(i)
        word_list = []
        for word_ in seg_list:
            word_list.append(word_)
        word_str = ' '.join(word_list)
        if len(word_list) == 0:
            hack_seg_list.append('NaN')
        else:
            hack_seg_list.append(word_str)
        hack_seg_list.append(len(word_list))
    else:
        hack_seg_list.append('NaN')
        hack_seg_list.append(0)

#news.columns = ['label_num','label_name','content']
#print(len(news))
hack_seg1 = pd.DataFrame(np.array(hack_seg_list).reshape(int(len(hack_seg_list)/2),2))
hack_seg1.columns = ['seg_word','word_count']

In [12]:
#有利原告編碼
hack_seg_list = []
analyer = PerceptronLexicalAnalyzer()
for i in df['有利原告']:
    if type(i) == str:
        seg_list = analyer.segment(i)
        word_list = []
        for word_ in seg_list:
            word_list.append(word_)
        word_str = ' '.join(word_list)
        if len(word_list) == 0:
            hack_seg_list.append('NaN')
        else:
            hack_seg_list.append(word_str)
        hack_seg_list.append(len(word_list))
    else:
        hack_seg_list.append('NaN')
        hack_seg_list.append(0)

#news.columns = ['label_num','label_name','content']
#print(len(news))
hack_seg2 = pd.DataFrame(np.array(hack_seg_list).reshape(int(len(hack_seg_list)/2),2))
hack_seg2.columns = ['seg_word','word_count']

In [13]:
#有利被告編碼
hack_seg_list = []
analyer = PerceptronLexicalAnalyzer()
for i in df['有利被告']:
    if type(i) == str:
        seg_list = analyer.segment(i)
        word_list = []
        for word_ in seg_list:
            word_list.append(word_)
        word_str = ' '.join(word_list)
        if len(word_list) == 0:
            hack_seg_list.append('NaN')
        else:
            hack_seg_list.append(word_str)
        hack_seg_list.append(len(word_list))
    else:
        hack_seg_list.append('NaN')
        hack_seg_list.append(0)

#news.columns = ['label_num','label_name','content']
#print(len(news))
hack_seg3 = pd.DataFrame(np.array(hack_seg_list).reshape(int(len(hack_seg_list)/2),2))
hack_seg3.columns = ['seg_word','word_count']

In [30]:
hack_seg_all = pd.concat([hack_seg['seg_word'],hack_seg1['seg_word'],hack_seg2['seg_word'],hack_seg3['seg_word']], axis = 1)
hack_seg_all.columns = ['原告主張','被告主張','有利原告','有利被告']

In [34]:
df = pd.concat([df,hack_seg_all],axis =1)

,學歷,年齡,數額,原告要求金額,最後判賠金額,原告主張,被告主張,有利原告,有利被告
0,1.954955,36.272727,1000000,800000,500000,服用 藥物 昏沈 不能 抗拒,NaN,趁 原告 服用 安眠藥 法抗拒 ， 事 後悔意 從未 道歉 ， 指稱 原告 設局 引誘 被告,NaN
1,2.500000,36.272727,52540,1300000,900000,因 酒醉 不知 不能 抗拒,當日 因 飲酒過量 精神 恍惚 ， 證人 之 證詞 互有 矛盾 ， DNA 未 為 23 種...,被告 利用 原告 酒醉 不知 不能 抗拒,NaN
2,1.954955,36.272727,36000,700000,50000,被告 不滿 原告 以 徒手 拍打 其 臀部 ， 而 徒手 反 推 原告 ， 原告 當場 倒地...,正當防衛 ， 傷害 故意 ， 被告 才是 受害人,NaN,NaN
3,1.954955,36.272727,1914876,300000,250000,被告 趁 原告 尋求 醫療 協助 之際 ， 要求 發生 性行為 ， 醫療 行為 不當 ， 致...,術前 已 盡 評估 之事 ， 原告 為 護士 ， 於 術前 應 已 了解 風險,趁 病人 不適 且 法 逃脫 醫師 治療 之 依賴 ， 違反 道德,NaN
4,2.500000,36.272727,0,1000000,400000,被告 明知 其 為 低 智商 而 強制 性交,二 人 互有 情愫 ， 並未 暴力 脅迫 原告 ， 經濟 能力 不 佳 ， 不應 請求 高額...,被告 明知 其 為 低 智商 而 強制 性交,NaN
5,2.000000,36.272727,0,800000,300000,因 酒醉 不知 不能 抗拒 ， 2 次 乘機 性交,證詞 前後 矛盾 ， 法 排除 合意 性交 之 可能,違反 原告 之 意願 2 次 乘機 性交,NaN
6,1.954955,36.272727,0,3000000,850000,被告 爬進 原告 房間 ， 趁 其 昏睡 強制 性交 ， 匿名 恐嚇 原告 不得 洩漏,被告 沒有 那麼 多 錢 ， 希望 賠償 金少 一點,原告 年紀 輕 ， 尚 在 追逐 夢想 年紀,NaN
7,1.954955,12.000000,1048450,400000,150000,A女 僅 小六 ， 長期 被 被告 猥褻 ， 致 患PTSD ， 期間 並 出現 腦中 有 ...,僅有 2 次 猥褻 行為,A女 於 事發 時 為 12 歲 之 少年 ， 身心 發育 尚 未成 熟,NaN
8,1.954955,20.000000,35000,2000000,80000,趁 其 進 房間 詢問 WIFI 密碼 之際 強制 性交 ， 造作 原告 林 女 之 父 、...,請求 駁回 ， 稱 其 此等 資力 負擔 賠償金,NaN,原告 案發 後 仍 與 被告 視訊 ， 原告 證詞 前後 矛盾
9,0.500000,36.272727,80000,800000,500000,因 酒醉 不知 不能 抗拒,並 趁機 性交,足 致 原告 身心 受到 極大 創傷,NaN


In [38]:
df[['學歷','年齡','數額']]

,學歷,年齡,數額
0,1.954955,36.272727,1000000
1,2.500000,36.272727,52540
2,1.954955,36.272727,36000
3,1.954955,36.272727,1914876
4,2.500000,36.272727,0
5,2.000000,36.272727,0
6,1.954955,36.272727,0
7,1.954955,12.000000,1048450
8,1.954955,20.000000,35000
9,0.500000,36.272727,80000


In [96]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import lightgbm as lgb
from sklearn.model_selection import train_test_split
x_train, x_test, \
x_train1, x_test1, \
x_train2, x_test2, \
x_train3, x_test3, \
x_train4, x_test4, \
y_train, y_test = train_test_split(df['原告主張'],df['被告主張'],df['有利原告'],df['有利被告'], df[['學歷','年齡','數額']],df['最後判賠金額'] ,test_size=0.1)
cw = lambda x: int(x)
x_train = x_train
y_train = np.array(y_train.apply(cw))
x_test = x_test
y_test = np.array(y_test.apply(cw))

In [117]:
 # 将语料转化为词袋向量，根据词袋向量统计TF-IDF
vectorizer = CountVectorizer(max_features=200)
tf_idf_transformer = TfidfTransformer()
tf_idf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(x_train))
x_train_weight = tf_idf.toarray()  # 训练集TF-IDF权重矩阵
tf_idf = tf_idf_transformer.transform(vectorizer.transform(x_test))
x_test_weight = tf_idf.toarray()  # 测试集TF-IDF权重矩阵

vectorizer1 = CountVectorizer(max_features=200)
tf_idf_transformer1 = TfidfTransformer()
tf_idf1 = tf_idf_transformer1.fit_transform(vectorizer1.fit_transform(x_train1))
x_train_weight1 = tf_idf1.toarray()  # 训练集TF-IDF权重矩阵
tf_idf1 = tf_idf_transformer1.transform(vectorizer1.transform(x_test1))
x_test_weight1 = tf_idf1.toarray()  # 测试集TF-IDF权重矩阵

vectorizer2 = CountVectorizer(max_features=200)
tf_idf_transformer2 = TfidfTransformer()
tf_idf2 = tf_idf_transformer2.fit_transform(vectorizer2.fit_transform(x_train2))
x_train_weight2 = tf_idf2.toarray()  # 训练集TF-IDF权重矩阵
tf_idf2 = tf_idf_transformer2.transform(vectorizer2.transform(x_test2))
x_test_weight2 = tf_idf2.toarray()  # 测试集TF-IDF权重矩阵

vectorizer3 = CountVectorizer(max_features=200)
tf_idf_transformer3 = TfidfTransformer()
tf_idf3 = tf_idf_transformer3.fit_transform(vectorizer3.fit_transform(x_train))
x_train_weight3 = tf_idf3.toarray()  # 训练集TF-IDF权重矩阵
tf_idf3 = tf_idf_transformer3.transform(vectorizer3.transform(x_test))
x_test_weight3 = tf_idf3.toarray()  # 测试集TF-IDF权重矩阵

x_train_all = np.concatenate((x_train_weight,x_train_weight1,x_train_weight2,x_train_weight3,np.array(x_train4)),axis=1)
x_test_all =np.concatenate((x_test_weight,x_test_weight1,x_test_weight2,x_test_weight3,np.array(x_test4)),axis=1)
# 创建成lgb特征的数据集格式
lgb_train = lgb.Dataset(np.array(x_train_all), y_train)
lgb_val = lgb.Dataset(np.array(x_test_all), y_test, reference=lgb_train)
param = {'boosting_type':'gbdt',
     'objective':'regression',

     'num_leaves':8, 
     'max_depth':10,
     'learning_rate':0.01,

     'metric': 'rmse',
     'subsample':1.0,
     'colsample_bytree':1.0,
     'n_estimators':100,
     'bagging_fraction': 1.0,
     'feature_fraction': 0.8,
     'min_child_samples':13,
     'reg_lambda': 0.08,
     'reg_alpha': 0

    }

gbm = lgb.train(param,lgb_train,valid_sets=lgb_val)

# 构建lightGBM模型
#     params = {'max_depth': 5, 'min_data_in_leaf': 20, 'num_leaves': 35,
#               'learning_rate': 0.1, 'lambda_l1': 0.1, 'lambda_l2': 0.2,
#               'objective': 'multiclass', 'num_class': 3, 'verbose': -1}
#     # 设置迭代次数，默认为100，通常设置为100+
#     num_boost_round = 1000
#     # 训练 lightGBM模型
#     gbm = lgb.train(params, lgb_train, num_boost_round, verbose_eval=100, valid_sets=lgb_val)

# 保存模型到文件
# gbm.save_model('data/lightGBM_model')

# 预测数据集
y_pred = gbm.predict(x_test_all, num_iteration=gbm.best_iteration)
y_pred

#label_all = ['负面', '中性', '正面']
#     confusion_mat = metrics.confusion_matrix(y_test, y_predict)
#     df = pd.DataFrame(confusion_mat, columns=label_all)
#     df.index = label_all

#     print('准确率：', metrics.accuracy_score(y_test, y_predict))
#     print('confusion_matrix:', df)
#     print('分类报告:', metrics.classification_report(y_test, y_predict))

[1]	valid_0's rmse: 492182
[2]	valid_0's rmse: 491931
[3]	valid_0's rmse: 491296
[4]	valid_0's rmse: 490687
[5]	valid_0's rmse: 489957
[6]	valid_0's rmse: 489378
[7]	valid_0's rmse: 488823
[8]	valid_0's rmse: 488292
[9]	valid_0's rmse: 488057
[10]	valid_0's rmse: 488163
[11]	valid_0's rmse: 487556
[12]	valid_0's rmse: 487369
[13]	valid_0's rmse: 486626
[14]	valid_0's rmse: 486796
[15]	valid_0's rmse: 486810
[16]	valid_0's rmse: 486277
[17]	valid_0's rmse: 486321
[18]	valid_0's rmse: 486380
[19]	valid_0's rmse: 485826
[20]	valid_0's rmse: 485714
[21]	valid_0's rmse: 485897
[22]	valid_0's rmse: 485255
[23]	valid_0's rmse: 485433
[24]	valid_0's rmse: 485503
[25]	valid_0's rmse: 485105
[26]	valid_0's rmse: 484931
[27]	valid_0's rmse: 485004
[28]	valid_0's rmse: 484847
[29]	valid_0's rmse: 484660
[30]	valid_0's rmse: 484693
[31]	valid_0's rmse: 484089
[32]	valid_0's rmse: 483997
[33]	valid_0's rmse: 483670
[34]	valid_0's rmse: 482551
[35]	valid_0's rmse: 482326
[36]	valid_0's rmse: 481244
[

D:\Anaconda\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


array([ 815781.5126153 ,  602208.51351334,  837089.97880378,
        608583.7241928 ,  651182.75057452,  576688.38018685,
        543425.4983638 ,  483423.58227059,  431058.38339141,
        648252.8494337 ,  849283.01669629,  447917.26249659,
        680212.38416277,  699600.5566159 ,  625818.1201917 ,
        815781.5126153 ,  424681.56446175, 1028003.98743561,
        513671.13571487,  710495.95231436,  522125.70129846])

In [109]:
# 交叉驗證找參數
num_round = 1000
cv_results = lgb.cv(param, lgb_train, num_round, nfold=5, stratified=False, shuffle=True, metrics='rmse',
    early_stopping_rounds=50, verbose_eval=50, show_stdv=True, seed=0)

print('best n_estimators:', len(cv_results['rmse-mean']))
print('best cv score:', cv_results['rmse-mean'][-1])

[50]	cv_agg's rmse: 804582 + 364749
best n_estimators: 24
best cv score: 800915.9939238336


D:\Anaconda\lib\site-packages\lightgbm\engine.py:430: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [111]:
from sklearn.model_selection import GridSearchCV
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=7,
                              learning_rate=0.01, n_estimators=42, max_depth=5,
                              metric='rmse', bagging_fraction = 1.0,feature_fraction = 0.8)

params_test1={'max_depth': range(3,8,1), 'num_leaves':range(5, 20, 1)}

gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test1, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)
gsearch1.fit(np.array(x_train_all), y_train)
# gsearch1.grid_scores_
print(gsearch1.best_params_)
print(gsearch1.best_score_)
print(gsearch1.best_estimator_)

Fitting 5 folds for each of 75 candidates, totalling 375 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 202 tasks      | elapsed:    2.5s
[Parallel(n_jobs=4)]: Done 375 out of 375 | elapsed:    3.2s finished
D:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'max_depth': 4, 'num_leaves': 5}
-786582340031.6552
LGBMRegressor(bagging_fraction=1.0, boosting_type='gbdt', class_weight=None,
              colsample_bytree=1.0, feature_fraction=0.8,
              importance_type='split', learning_rate=0.01, max_depth=4,
              metric='rmse', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=42, n_jobs=-1, num_leaves=5,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)


In [118]:
y_pred

array([ 815781.5126153 ,  602208.51351334,  837089.97880378,
        608583.7241928 ,  651182.75057452,  576688.38018685,
        543425.4983638 ,  483423.58227059,  431058.38339141,
        648252.8494337 ,  849283.01669629,  447917.26249659,
        680212.38416277,  699600.5566159 ,  625818.1201917 ,
        815781.5126153 ,  424681.56446175, 1028003.98743561,
        513671.13571487,  710495.95231436,  522125.70129846])

In [119]:
y_test

array([2300000,  700000, 1000000,  800000, 1000000, 1000000,  250000,
        200000,  350000,  200000,  750000,  250000,  900000,  100000,
        900000,  450000,  200000,  300000,  600000,  600000,   50000],
      dtype=int64)

In [120]:
error = []
for i in range(len(y_test)):
    error.append(y_test[i] - y_pred[i])
    
#print("Errors: ", error)
#print(error)
squaredError = []
absError = []
for val in error:
    squaredError.append(val * val)#平方
    absError.append(abs(val))#誤差絕對值
    
#print("Square Error: ", squaredError)
#print("Absolute Value of Error: ", absError)
print("MSE = ", sum(squaredError) / len(squaredError))#平均平方誤差MSE


#from math import sqrt
#print("RMSE = ", sqrt(sum(squaredError) / len(squaredError)))#平均平方根誤差RMSE
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE

MSE =  211395533693.27075
MAE =  342514.922258945
